# Black cats MIA 🐱‍👤

## Game 107379149

<img src="pics/discord_game_summary.png" alt="discord_game_summary" width="300"/>

This was a game that took place between Player MunkFSH (I) and yoifuro on Sept 3rd around 8am EDT. This was our 5th game in our Season 53 League D3 match.

Although it lasted only 11 turns, this board was rather complex due to a couple key interactions.
- Remake - Poor House
  - An annoying interaction between the two since we would like to use remake to get rid of coppers from our deck. Due to the presence of poor house, the remake would turn coppers into poor houses making it signficantly worse as a trasher.
- Black cat - Way of the frog
  - If we know in advance that it is very likely that our opponent will buy a green card in the following turn (for example right after securing a vp lead), we can use way of the frog to top-deck black cats as a way to attack our opponent or dissuade them from purchasing green cards.
- Mastermind - Way of the frog
  - Similar to the previous interaction we can use way of the frog to topdeck cards we have in our hand now, that we would like to mastermind in the following turn. Ideally you would want to mastermind a draw card like black cat or hunting grounds due to the lack of trashing on the board.
  
I made an interesting play on turn 9 (decision number 128) that I would like to investigate further.

## An Interesting Decision

<img src="pics/main_decision_point.png" alt="discord_game_summary" width="1000"/>

Here I played the remake to try to score some points. Because I remade my estates earlier into silvers and anvils I was down 3 points to my opponent. I needed to score at least 4 points this turn to take the lead. I was planning on converting the poor house --> estate, converting the wandering minstrel --> duchy, and purchasing a province giving myself a 7-point lead over my opponent. This would then let me top-deck 2 black cats via way of the frog effectively giving myself a 9-point lead over my opponent. I knew my opponent had 5 black cats in his deck (I had the remaining 5). The previous turn my opponent had triggered a fresh shuffle without top-decking any black cats. I figured if I could end the turn taking 3 curses or fewer, I would have a pretty good chance at finishing the game by piling anvils.

If my opponent played 3 black cats exactly, they would have 11 cards in hand with a good chance they can province + duchy, but not more than that. They would receive 2 curses from my cats leading to even points. This would lead to a tie if I can end the game on my turn.

Any fewer black cats and I would likely win due to the point advantage. Any more cats and I would lose due to the point deficit.

In the actual game my opponent had no black cats in their hand on this turn and I proceeded to win off of my point lead & by piling the anvils. This was the best case scenario for me, but this outcome was far from guaranteed. It was not a safe play, but it did get my statistical brain thinking. 
- How likely was this scenario? 
- How likely was the event that I would receive 2 curses or less on turn 9? 
- How risky was my play on turn 9? 
- How did I evaluate such a complex board state in less than a minute? 
- And did I make a mistake during my rapid calculations?

## How likely was my opponent to not have any black cats in their starting hand? 🐱‍👤

This is a simple probability problem.

My opponent has 25 cards in their deck total, and they draw 5 at the end of their turn. They did not use way of the frog to topdeck any black cats, and they had no cards in their drawpile (triggered a fresh shuffle).

Going through the 5 cards they draw one-by-one treating each draw as an independent event we get the following probabilities for avoiding a black cat as: 
- $\frac{20}{25}$ for the first draw (5 🐈's in deck)
- $\frac{19}{24}$ for the second draw (5 🐈's in deck, but we took a non-cat out)
- $\frac{18}{23}$ for the third draw
- $\frac{17}{22}$ for the fourth draw
- $\frac{16}{21}$ for the fifth draw
Since these events are independent of each other we can find their joint probability by multiplying these fractions together
$$\frac{20*19*18*17*16}{25*24*23*22*21}$$

In [5]:
from functools import reduce
reduce(lambda x,y: x*y,[(i-5)/i for i in range(25,20,-1)])

0.29181253529079615

There's about a 29.2% chance that my opponent did not have any black cats in their starting hand giving their previous turn play.

## How likely was the event that I would receive 2 curses or less on turn 9?

To calculate this probability it is a bit tricker as each black cat can dig a further 2 cards deeper into the deck. In fact if we wanted to enumerate all the possibilities to search through, we would have to look at the first 13 cards of my opponent's deck as the 1st black cat could draw the 6th & 7th cards, the second black cat can draw the 8th & 9th cards, the third black cat can draw the 10th & 11th cards, and the 4th black cat can draw the 12th and 13th cards (the 5th black cat will draw an additional 2 cards but at that point those cards don't matter since my opponent found all their black cats).

Enumerating all the possibilities of 13 initial cards is huge!

$$25 *24 *23 *22 *21 *20 *19 *18 *17 *16 *15 *14 *13$$

In [9]:
reduce(lambda x,y: x*y,[i for i in range(25,25-13,-1)])

32382376266240000

That's 32,382,376,266,240,000! I have no way of enumerating all those possibilities, but they might not all matter. In fact when my opponent runs out cats the order of the remaining 13 cards doesn't really matter, so we can make use of some early termination principles to try to make this probability easier to calculate. Specifically I will start by naming the 6 important outcomes that my opponent might react with:
- 0 black cats
- 1 black cat
- 2 black cats
- 3 black cats
- 4 black cats
- 5 black cats

Going through each of the 6 cases:
- 0 black cats
  - There are $(20*19*18*17*16) * 20 * 19 * 18 * 17 * 16 * 15 * 14 * 13$ options for a 13-card starting deck to result in 0 black cats in the first 5-card hand.
- 1 black cat
  - $5*(5*20*19*18*17)* 16 * 15 * 18 * 17 *16 * 15 * 14 * 13$
- 2 black cats
  - $10*(5*4*20*19*18)*17*16*15*14 16*15*14*13 +$
  - $5*(5*20*19*18*17)*2*(4*16)*15*14*16*15*14*13$
- 3 black cats

### MC simulations to verify math

In [7]:
# perform MC simulations to verify the probabilities & counts
from random import shuffle

def repeated_sample_without_replacement(my_list, n):
    shuffle(my_list)
    return my_list[:n]

opp_deck = list(range(25)) # cats are 0,1,2,3,4

seed = 19890417
sims = int(1e6)

def count_cats(deck):
    # deck is list of ints [0,24]
    # cats are [0-4]
    cats = 0
    draw = 5
    i = 0
    while draw > 0:
        if deck[i] <= 4:
            cats+=1
            draw+=2
        draw-=1
        i+=1
    return cats

# some test cases
assert(count_cats([24,23,22,21,20]) == 0)
assert(count_cats([24,23,22,21,0,20,19]) == 1)
assert(count_cats(list(range(25))) == 5)

# run sims
results = [count_cats(repeated_sample_without_replacement(opp_deck,15)) for sim in range(sims)]

# coallate_results
from collections import Counter
results_dict = Counter(results)

import pandas as pd
results_df = pd.DataFrame([{"cats":i, "counts":results_dict[i], "%":results_dict[i]/sims} for i in range(5)])
results_df

,cats,counts,%
0,0,291586,0.291586
1,1,288073,0.288073
2,2,210897,0.210897
3,3,128051,0.128051
4,4,62519,0.062519


## How did I evaluate such a complex board state in less than a minute? ⏱

My rational was that 5 black cats in 25 cards, I expect my opponent to have at least 1, maybe 2, very lucky if 3+ in their opening hand. I put my odds of receiving 2 curses or less at ~80%. This was not empirically driven, but in trying to apply quick gameplay reasoning... The first 5 cards can be estimated to be random samples from a binomial(p=0.2) which has an expected value of 1 with variance of 0.8. Within 2 standard deviations, we can expect to see about 2 black cats. Factoring in arithmetic errors, I gave this play about an 80% chance of working the way I wanted it to.